# Predict Water Amount Needed

## Get and Prepare Weather Data from DataBase

In [11]:
import pymysql.cursors
import pandas as pd
import numpy as np

connection = pymysql.connect(host = "mikmak.cc", user="sensor", passwd="Gaffe2017", db="weatherDW")
query = ('SELECT * FROM log_v_last24Hours WHERE S_Text <> "None"')

with connection.cursor() as cursor:
    cursor.execute(query)
connection.commit()
e_Log = cursor.fetchall()
connection.close()

e_Log = (np.array(e_Log))

144

In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

t_Log = e_Log

t_Log = pd.DataFrame(data = t_Log[1:,:],
                     index = t_Log[1:,2],
                     columns =["ID","Value","TimeStamp"] )


t_Log = t_Log.pivot(index='TimeStamp', columns='ID', values='Value')

t_Log.info()

t_Log = t_Log.sort_index(ascending = True) #It should be ascending, for rolling calculation later
t_Log = t_Log.astype(float)
t_Log = t_Log.dropna(how = "any")

t_Log

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25 entries, 2017-04-04 01:49:45 to 2017-04-05 00:49:50
Data columns (total 6 columns):
Humidity       24 non-null object
Pressure       24 non-null object
Rain           24 non-null object
Sunshine       24 non-null object
Temperature    24 non-null object
Wind           23 non-null object
dtypes: object(6)
memory usage: 1.4+ KB


ID,Humidity,Pressure,Rain,Sunshine,Temperature,Wind
TimeStamp,,,,,,
2017-04-04 01:49:45,83.0,964.8,0.0,0.0,10.9,4.0
2017-04-04 02:49:44,88.0,964.5,0.0,0.0,10.1,0.4
2017-04-04 03:49:45,83.0,964.2,0.0,0.0,10.5,2.9
2017-04-04 04:49:44,84.0,964.1,0.0,0.0,10.9,4.7
2017-04-04 05:49:44,90.0,964.2,0.1,0.0,10.1,5.4
2017-04-04 06:49:44,91.0,964.3,0.0,0.0,9.4,2.5
2017-04-04 07:49:45,90.0,964.5,0.0,0.0,9.7,1.8
2017-04-04 08:49:44,88.0,964.7,0.0,0.0,10.1,0.7
2017-04-04 09:49:47,85.0,964.6,0.0,8.0,11.4,5.8


## Transform Data into Input Vector

### Order Data and Calculate Means and Sums

In [13]:
X = pd.DataFrame()

lastRow = len(t_Log.axes[0]) - 1

X = X.assign(tre200b0=t_Log.Temperature[[lastRow]])
X = X.assign(ure200b0=t_Log.Humidity[[lastRow]])
X = X.assign(rre200b0=t_Log.Rain[[lastRow]])
X = X.assign(sre000b0=t_Log.Sunshine[[lastRow]])
X = X.assign(fu3010b0=t_Log.Wind[[lastRow]])
X = X.assign(prestab0=t_Log.Pressure[[lastRow]])
#"tre200b0","ure200b0","rre150b0","sre000b0","fu3010b0","prestab0"

t_LogRolling = t_Log.rolling(len(t_Log.axes[0]))

X = X.assign(tre200b0_mean=t_LogRolling.Temperature.mean())
X = X.assign(ure200b0_mean=t_LogRolling.Humidity.mean())
X = X.assign(rre200b0_sum=t_LogRolling.Rain.sum())
X = X.assign(sre000b0_sum=t_LogRolling.Sunshine.sum())
X = X.assign(fu3010b0_mean=t_LogRolling.Wind.mean())
X = X.assign(prestab0_mean=t_LogRolling.Pressure.mean())

X.transpose()

TimeStamp,2017-04-04 23:49:49
tre200b0,9.200000
ure200b0,78.000000
rre200b0,0.300000
sre000b0,0.000000
fu3010b0,14.400000
prestab0,964.800000
tre200b0_mean,11.368182
ure200b0_mean,71.954545
rre200b0_sum,0.400000
sre000b0_sum,34.000000


### Poly Features etc.

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import PolynomialFeatures

polyDegree = 5
poly = PolynomialFeatures(degree=polyDegree)
X = poly.fit_transform(X).astype(int)

poly.n_output_features_ 

X

6188

array([[              1,               9,              78, ...,
            23704286624,   4446681049857, 834151757968067]])

## Classify!
Use Logistic Regression Modell to decide if 
- no water at all or 
- a lot of water or 
- normal amount of water
is needed

### Load Model

In [15]:
from sklearn.externals import joblib
model = joblib.load('plant.Watering.data/logreg_5deg.pkl')

### Do Calculation and Show Result
- 0 - means a lot of water
- 1 - is medium
- 2 - device should stay off

In [17]:
result = model.predict(X)[0]

from sklearn import preprocessing
le = joblib.load('plant.Watering.data/leModel.pkl')
le.inverse_transform(result)


'Med'